In [1]:
%matplotlib inline
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import pickle

In [11]:
in_path = "test_v2.csv" #Path where the large file is
#out_path = "" #Path to save the pickle files to
chunk_size = 30000 #size of chunks relies on your available memory
separator = ","

JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']

reader = pd.read_csv(in_path,sep=separator,chunksize=chunk_size,
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'},
                    low_memory=False,error_bad_lines=False)    


for i, chunk in enumerate(reader):
    out_file = "test/test_{}.pkl".format(i+1)
    with open(out_file, "wb") as f:
        pickle.dump(chunk,f,pickle.HIGHEST_PROTOCOL)

In [12]:
import glob
pickle_path = "test" #Same Path as out_path i.e. where the pickle files are

data_p_files=[]
for name in glob.glob(pickle_path + "/test_*.pkl"):
    data_p_files.append(name)


test = pd.DataFrame([])
for i in range(len(data_p_files)):
    test = test.append(pd.read_pickle(data_p_files[i]),ignore_index=True)

In [14]:
test = test.drop(['hits', 'customDimensions'], axis=1)

In [15]:
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
for column in JSON_COLUMNS:
    column_as_df = json_normalize(test[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    test = test.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
print(f"Extracted test. Shape: {test.shape}")

Extracted test. Shape: (401589, 57)


In [16]:
test.head(2)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,device.browserVersion,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20180511,7460955084541987166,Not Socially Engaged,1526099341,2,1526099341,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not provided),organic,(not set),google
1,Direct,20180511,460252456180441002,Not Socially Engaged,1526064483,166,1526064483,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct)


In [23]:
# select constant columns
const_cols = [c for c in test.columns if test[c].nunique(dropna=False)==1 ]
const_cols

['socialEngagementType',
 'device.browserSize',
 'device.browserVersion',
 'device.flashVersion',
 'device.language',
 'device.mobileDeviceBranding',
 'device.mobileDeviceInfo',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.operatingSystemVersion',
 'device.screenColors',
 'device.screenResolution',
 'geoNetwork.cityId',
 'geoNetwork.latitude',
 'geoNetwork.longitude',
 'geoNetwork.networkLocation',
 'totals.visits',
 'trafficSource.adwordsClickInfo.criteriaParameters']

In [24]:
test = test.drop(const_cols, axis=1)

In [25]:
test.shape

(401589, 38)

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401589 entries, 0 to 401588
Data columns (total 38 columns):
channelGrouping                                 401589 non-null object
date                                            401589 non-null int64
fullVisitorId                                   401589 non-null object
visitId                                         401589 non-null int64
visitNumber                                     401589 non-null int64
visitStartTime                                  401589 non-null int64
device.browser                                  401589 non-null object
device.deviceCategory                           401589 non-null object
device.isMobile                                 401589 non-null bool
device.operatingSystem                          401589 non-null object
geoNetwork.city                                 401589 non-null object
geoNetwork.continent                            401589 non-null object
geoNetwork.country                         

In [27]:
test.sample(2)

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,device.deviceCategory,device.isMobile,device.operatingSystem,...,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
59918,Direct,20180725,0356436964905854241,1532508966,1,1532508966,Chrome,desktop,False,Windows,...,NaN,NaN,NaN,NaN,(not set),True,(not set),(none),(not set),(direct)
107158,Organic Search,20181004,142139349729638574,1538685758,1,1538685758,Safari,desktop,False,Macintosh,...,NaN,NaN,NaN,NaN,(not set),NaN,(not provided),organic,(not set),google


In [28]:
test.to_pickle("test_clean.pkl")